In [ ]:
#IMPORTING TENSORFLOW LIBRARY

import tensorflow as tf 
print(tf.__version__)

2.2.0


In [ ]:
#IMPORTING THE IMDB REVIEWS DATASET FROM TFDS(Tensorflow datasets)

import tensorflow_datasets as tfds 
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True )

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete012GEL/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete012GEL/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete012GEL/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
#IMPORTING NUMPY(Used for working with arrays)

import numpy as np

#GETTING THE TRAINING DATA(Data on which we train the model) AND TESTING DATA(Data on which we evaluate the performance of our model)

train_data,test_data=imdb['train'],imdb['test']

training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

#SPLITTING THE SENTENCES(Reviews) AND THEIR LABELS(Binary ,i.e,positive or negative?)

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)




In [ ]:
vocabulary=10000
oov='<<OOV>>'
max_length=120

#SETTING UP THE TOKENIZER(Giving tokens to words ,to feed in into the neural network) AND PADDING THEM(So that they have same length)

from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocabulary,oov_token=oov)
tokenizer.fit_on_texts(training_sentences)

#WORD INDEX TELLS US WHICH NUMBER IS ASSIGNED TO WHICH PARTICULAR WORD

word_index=tokenizer.word_index 
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating='post')

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)





In [ ]:
#CREATING A FUNTION TO GET THE WORDS FOR THE CORRESPONDING TOKENS

rev_word_index=dict([(value,key)for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([rev_word_index.get(i,'?') for i in text])

print(training_sentences[3])
print(decode_review(padded[3]))

This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <<OOV>> into a big arm chair and <<OOV>> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <<OOV>> to cross no dangerous waters just a warm and witty <<OOV>> through new york life at its best a family film in every sense and one that deserves the praise it received


In [ ]:
#DESIGNING A DEEP NEURAL NETWORK HAVING SEVERAL LAYERS 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding

model=Sequential()

#EMBEDDING LAYER

model.add(Embedding(10000,16,input_length=120))

#FLATTEN LAYER(To make it into a single dimension so that we can feed it to Dense Layer)

model.add(Flatten())

#DENSE LAYER

model.add(Dense(6,activation='relu'))

#FINAL DENSE LAYER(Which predicts in Binary terms)

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#TRAINING THE NEURAL NETWORK FOR 10 EPOCHS 

model.fit(padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4973 - accuracy: 0.7407 - val_loss: 0.3389 - val_accuracy: 0.8530
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.2375 - accuracy: 0.9092 - val_loss: 0.3749 - val_accuracy: 0.8375
Epoch 3/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0860 - accuracy: 0.9790 - val_loss: 0.4450 - val_accuracy: 0.8304
Epoch 4/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0222 - accuracy: 0.9973 - val_loss: 0.5220 - val_accuracy: 0.8333
Epoch 5/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0064 - accuracy: 0.9995 - val_loss: 0.5891 - val_accuracy: 0.8260
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.6305 - val_accuracy: 0.8322
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 9.3381e-04 - accuracy: 1.0000 - val_loss: 0.6736 - val_accuracy: 0.8317
Ep

In [ ]:
#FOR GETTING THE WEIGHTS OF THE LAYER(So they we can write the embeddings to vector file later)

e=model.layers[0]
weights=e.get_weights()[0]


In [ ]:
print(weights.shape)

(10000, 16)


In [ ]:
import io

#CREATING TWO EMPTY FILES(To store vectors and metadata in order to view the projections in embedding projector)
out_v=io.open('vec.tsv','w',encoding='utf-8')
out_m=io.open('met.tsv','w',encoding='utf-8')

#ARRAY TO WRITE THE DATA INTO THE FILES CREATED ABOVE

for word_num in range(1,vocabulary):
  actual_word=rev_word_index[word_num]
  out_m.write(actual_word+ '\n')
  embeddings=weights[word_num]
  out_v.write('\t'.join([str(x) for x in embeddings])+ '\n')

out_v.close()
out_m.close()





In [ ]:
#DOWNLOADING THE FILES 
from google.colab import files 
files.download('vec.tsv')
files.download('met.tsv')

#THESE TWO FILES ARE SIMPLY TO BE UPLOADED TO THE EMBEDDING PROJECTOR(To get the projections where words having similar meaning are grouped together)